## 4. Ingestão CDC via Streaming

### Modos de Execução

**Modo Atual: `availableNow=True`**
- Processa todos os dados disponíveis e para automaticamente
- Ideal para execução via Jobs/Pipelines
- Sem custo de cluster ocioso

**Modo Contínuo (comentado abaixo):**
- Mantém o streaming ativo indefinidamente
- Processa novos arquivos conforme chegam
- Requer cluster sempre ligado

# Pipeline de Ingestão CDC - Upcell

Este notebook implementa o pipeline de ingestão de dados CDC (Change Data Capture) do S3 para o Bronze no Databricks.

## Objetivo
- Full-load: Carga inicial completa das tabelas
- CDC: Ingestão incremental com operações Insert, Update e Delete
- Delta Lake: Merge atômico na camada Bronze

## Requisitos
- Tabelas no S3: `s3://meudatalake-raw/upcell/`
- Catálogo: `bronze.upcell`
- Coluna de controle: `DtAtualizacao` (presente em todos os arquivos)

## 1. Importações e Setup

In [0]:
import delta
def table_exists(catalog, database, table):
    count = (spark.sql(f"SHOW TABLES IN `{catalog}`.`{database}`")
               .filter(f"database = '{database}' AND tableName = '{table}'")
               .count())
    return count == 1

In [0]:
catalog = "bronze"
schema = "upcell"

tablename = "produtos"
id_field = "IdProduto"
timefield = "DtAtualizacao"

In [0]:
# Captura o schema dos arquivos CDC para usar no streaming
df_sample = spark.read.format("parquet").load(f"/Volumes/raw/upcell/cdc/{tablename}/")
df_schema = df_sample.schema
print(f"Schema capturado: {len(df_schema.fields)} colunas")

In [0]:
if not table_exists(catalog, schema, tablename):
    print("tabela nao existe")
    df_full = spark.read.format("parquet").load(f"/Volumes/raw/upcell/full-load/{tablename}/")
    
    (df_full.coalesce(1)
        .write
        .format("delta")
        .mode("overwrite")
        .saveAsTable(f"{catalog}.{schema}.{tablename}"))
else:
    print("tabela ja existe")


## 2. Configuração e Full-Load

Define variáveis de configuração e cria a tabela inicial se não existir.

In [0]:
bronze = delta.DeltaTable.forName(spark, f"{catalog}.{schema}.{tablename}")

In [0]:
def upsert(df, deltatable):
    df.createOrReplaceGlobalTempView(f"view_{tablename}")

    query = f"""
    SELECT *
    FROM global_temp.view_{tablename}
    QUALIFY ROW_NUMBER() OVER (PARTITION BY {id_field} ORDER BY {timefield} DESC) = 1
    """

    df_cdc = spark.sql(query)

    (deltatable.alias("b")
        .merge(df_cdc.alias("d"), f"b.{id_field} = d.{id_field}")
        .whenMatchedDelete(condition = "d.OP = 'D'")
        .whenMatchedUpdateAll(condition = "d.OP = 'U'")
        .whenNotMatchedInsertAll(condition = "d.OP = 'I' OR d.OP = 'U'")
        .execute()
    )

df_stream = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .schema(df_schema)
    .load(f"/Volumes/raw/upcell/cdc/{tablename}/")
)

stream = (df_stream.writeStream
    .option("checkpointLocation", f"/Volumes/raw/upcell/cdc/{tablename}_checkpoint/")
    .foreachBatch(lambda df, batchID: upsert(df, bronze))
    
)


In [0]:
stream = stream.start()